In [1]:
import librosa
import numpy as np
import torch
import torch.nn as nn

In [2]:


class LSTMClassifier(nn.Module):
    def __init__(self, input_size=20, hidden_size=128, num_layers=3, num_classes=8, dropout=0.3):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=dropout,
                           bidirectional=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, (hn, _) = self.lstm(x)       
        logits = self.fc(hn[-1])          
        return logits

In [3]:
model_path = '../models/lstm_genre_classifier_with_class_weights.pth'

In [4]:
model = LSTMClassifier(input_size=20, hidden_size=128, num_layers=3, num_classes=8, dropout=0.3)
model.load_state_dict(torch.load(model_path, map_location='cpu'))
model.eval()

LSTMClassifier(
  (lstm): LSTM(20, 128, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  (fc): Linear(in_features=128, out_features=8, bias=True)
)

In [5]:
genre_id_to_name = {
    0: 'Electronic',
    1: 'Experimental',
    2: 'Folk',
    3: 'Hip-Hop',
    4: 'Instrumental',
    5: 'International',
    6: 'Pop',
    7: 'Rock'
}


In [6]:
def extract_mfcc_from_file(file_path, sample_rate=22050, n_mfcc=20, max_len=130):
    try:
        signal, sr = librosa.load(file_path, sr=sample_rate)
        mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc)

        if mfcc.shape[1] < max_len:
            pad_width = max_len - mfcc.shape[1]
            mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfcc = mfcc[:, :max_len]

        return mfcc.T  # shape: (130, 20)
    except Exception as e:
        print(f"Failed to process {file_path}: {e}")
        return None

In [7]:
def predict_genre(file_path):
    mfcc = extract_mfcc_from_file(file_path)
    if mfcc is None:
        return

    x = torch.tensor(mfcc, dtype=torch.float32).unsqueeze(0)  # Add batch dim: (1, 130, 20)
    with torch.no_grad():
        output = model(x)
        predicted_class = torch.argmax(output, dim=1).item()
        print(f"Predicted genre: {genre_id_to_name[predicted_class]}")

In [ ]:
predict_genre('../data/songs/Folk/AthensPierceMurphy.mp3')

Failed to process ../data/songs/Folk/Athens_-_Pierce_Murphy.mp3: [Errno 2] No such file or directory: '../data/songs/Folk/Athens_-_Pierce_Murphy.mp3'


C:\Users\ranuL\AppData\Local\Temp\ipykernel_12556\4119836396.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sr = librosa.load(file_path, sr=sample_rate)
c:\Users\ranuL\Documents\GitHub\GenreWave\venv\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [ ]:
predict_genre(
    '../data/songs/Folk/AcousticAstronautDestructionMillbyAcousticAstronaut.mp3')

Failed to process ../data/songs/Folk/Acoustic_Astronaut_-_Destruction_Mill_©2023_by_Acoustic_Astronaut™.mp3: [Errno 2] No such file or directory: '../data/songs/Folk/Acoustic_Astronaut_-_Destruction_Mill_©2023_by_Acoustic_Astronaut™.mp3'


C:\Users\ranuL\AppData\Local\Temp\ipykernel_12556\4119836396.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sr = librosa.load(file_path, sr=sample_rate)


In [ ]:
predict_genre('../data/songs/Folk/WalkTonightCharlieMosbrook.mp3')

Failed to process ../data/songs/Folk/Walk_Tonight_-_Charlie_Mosbrook_(2).mp3: [Errno 2] No such file or directory: '../data/songs/Folk/Walk_Tonight_-_Charlie_Mosbrook_(2).mp3'


C:\Users\ranuL\AppData\Local\Temp\ipykernel_12556\4119836396.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sr = librosa.load(file_path, sr=sample_rate)


In [ ]:
predict_genre('../data/songs/Rock/JohnLennonImagineMix.mp3')

Failed to process ../data/songs/Rock/059. John Lennon - Imagine (2010 Mix).mp3: [Errno 2] No such file or directory: '../data/songs/Rock/059. John Lennon - Imagine (2010 Mix).mp3'


C:\Users\ranuL\AppData\Local\Temp\ipykernel_12556\4119836396.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sr = librosa.load(file_path, sr=sample_rate)


In [ ]:
predict_genre('../data/songs/Rock/DonMcLeanAmericanPie.mp3')

Failed to process ../data/songs/Rock/060. Don McLean - American Pie.mp3: [Errno 2] No such file or directory: '../data/songs/Rock/060. Don McLean - American Pie.mp3'


C:\Users\ranuL\AppData\Local\Temp\ipykernel_12556\4119836396.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sr = librosa.load(file_path, sr=sample_rate)


In [ ]:
predict_genre("../data/songs/Rock/TheZombiesShesNotThere.mp3")

Failed to process ../data/songs/Rock/099. The Zombies - She's Not There.mp3: [Errno 2] No such file or directory: "../data/songs/Rock/099. The Zombies - She's Not There.mp3"


C:\Users\ranuL\AppData\Local\Temp\ipykernel_12556\4119836396.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sr = librosa.load(file_path, sr=sample_rate)


In [ ]:
predict_genre("../data/songs/Rock/DavidBowieChangesRemaster.mp3")

Failed to process ../data/songs/Rock/006. David Bowie - Changes (2015 Remaster).mp3: [Errno 2] No such file or directory: '../data/songs/Rock/006. David Bowie - Changes (2015 Remaster).mp3'


C:\Users\ranuL\AppData\Local\Temp\ipykernel_12556\4119836396.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sr = librosa.load(file_path, sr=sample_rate)
